# Projeto
## Sistema de recomendação

O mundo inteiro parece estar ouvindo sobre suas novas habilidades incríveis para analisar big data e construir sistemas úteis para eles! Você acabou de assinar um novo contrato com uma nova empresa de entrega de comida online. Esta empresa está tentando se diferenciar, recomendando novas refeições aos clientes com base nas preferências de outros clientes.

Você pode construir um sistema de recomendação para eles?

Seu resultado final deve estar na forma de uma função que pode pegar em um Spark DataFrame das classificações de um único cliente para várias refeições e gerar suas 3 principais refeições sugeridas. Por exemplo:

Boa sorte!

** * Nota de José: Eu inventei completamente esses dados alimentares, então é provável que as recomendações em si não façam nenhum sentido. Mas você deve obter uma saída semelhante ao que obtive com o exemplo de dataframe do cliente * **

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv('movielens_ratings.csv')

In [9]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
movieId,1501.0,49.405730,28.937034,0.0,24.0,50.0,74.0,99.0
rating,1501.0,1.774151,1.187276,1.0,1.0,1.0,2.0,5.0
userId,1501.0,14.383744,8.591040,0.0,7.0,14.0,22.0,29.0


In [10]:
df.corr()

,movieId,rating,userId
movieId,1.000000,0.036569,0.003267
rating,0.036569,1.000000,0.056411
userId,0.003267,0.056411,1.000000


In [11]:
import numpy as np
df['mealskew'] = df['movieId'].apply(lambda id: np.nan if id > 31 else id)

In [12]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
movieId,1501.0,49.405730,28.937034,0.0,24.0,50.0,74.0,99.0
rating,1501.0,1.774151,1.187276,1.0,1.0,1.0,2.0,5.0
userId,1501.0,14.383744,8.591040,0.0,7.0,14.0,22.0,29.0
mealskew,486.0,15.502058,9.250634,0.0,7.0,15.0,23.0,31.0


In [13]:
mealmap = { 2. : "Chicken Curry",   
           3. : "Spicy Chicken Nuggest",   
           5. : "Hamburger",   
           9. : "Taco Surprise",  
           11. : "Meatloaf",  
           12. : "Ceaser Salad",  
           15. : "BBQ Ribs",  
           17. : "Sushi Plate",  
           19. : "Cheesesteak Sandwhich",  
           21. : "Lasagna",  
           23. : "Orange Chicken",
           26. : "Spicy Beef Plate",  
           27. : "Salmon with Mashed Potatoes",  
           28. : "Penne Tomatoe Pasta",  
           29. : "Pork Sliders",  
           30. : "Vietnamese Sandwich",  
           31. : "Chicken Wrap",  
           np.nan: "Cowboy Burger",   
           4. : "Pretzels and Cheese Plate",   
           6. : "Spicy Pork Sliders",  
           13. : "Mandarin Chicken PLate",  
           14. : "Kung Pao Chicken",
           16. : "Fried Rice Plate",  
           8. : "Chicken Chow Mein",  
           10. : "Roasted Eggplant ",  
           18. : "Pepperoni Pizza",  
           22. : "Pulled Pork Plate",   
           0. : "Cheese Pizza",   
           1. : "Burrito",   
           7. : "Nachos",  
           24. : "Chili",  
           20. : "Southwest Salad",  
           25.: "Roast Beef Sandwich"}

In [14]:
df['meal_name'] = df['mealskew'].map(mealmap)

In [15]:
df.to_csv('Meal_Info.csv',index=False)

In [16]:
from pyspark.sql import SparkSession

In [17]:
spark = SparkSession.builder.appName('recconsulting').getOrCreate()

In [18]:
spark

In [19]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [27]:
data = spark.read.csv('Meal_Info.csv',inferSchema=True,header=True)

In [22]:
(training, test) = data.randomSplit([0.8, 0.2])

In [33]:
# Construir o modelo de recomendação usando ALS nos dados de treinamento

als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="mealskew", ratingCol="rating")

In [34]:
model = als.fit(training)

Py4JJavaError: An error occurred while calling o271.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 15.0 failed 1 times, most recent failure: Lost task 0.0 in stage 15.0 (TID 11, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$4: (double) => int)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:440)
	at scala.collection.Iterator$JoinItera

In [24]:
# Avalie o modelo calculando o RMSE nos dados de teste
predictions = model.transform(test)

predictions.show()

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

NameError: name 'model' is not defined